In [1]:
#xor异或, 2个输入中有一个为1但不全为1，结果是1，其他情况0。线性不可分
# 不使用placeholder的代码
# https://aimatters.wordpress.com/2016/01/16/solving-xor-with-a-neural-network-in-tensorflow/
# https://github.com/StephenOman/TensorFlowExamples/blob/master/xor%20nn/xor_nn.py

#  
# http://blog.csdn.net/u011026968/article/details/72084273
  
# https://gist.github.com/pannous/2b8e2e05cf05a630b132
# https://github.com/techdisrupt/XOR/blob/master/xor.py
import tensorflow as tf

In [3]:
Theta1 = tf.Variable(tf.random_uniform([2,2], -1, 1), name = "Theta1")
Bias1 = tf.Variable(tf.zeros([2]), name = "Bias1")

Theta2 = tf.Variable(tf.random_uniform([2,1], -1, 1), name = "Theta2")
Bias2 = tf.Variable(tf.zeros([1]), name = "Bias2")

In [4]:
def inputs():
    """输入"""
    XOR_X = [[0,0],[0,1],[1,0],[1,1]]
    XOR_Y = [[0],[1],[1],[0]]
    return tf.to_float(XOR_X), tf.to_float(XOR_Y) 

In [26]:
def combine_inputs(X):
    return tf.matmul(X,W) + b

def inference(X):
    """推断模型
    加入了隐藏层
    """
    with tf.name_scope("layer2") as scope:
        A2 = tf.sigmoid(tf.matmul(X, Theta1) + Bias1)

    with tf.name_scope("layer3") as scope:
        Hypothesis = tf.sigmoid(tf.matmul(A2, Theta2) + Bias2)
        
    return Hypothesis

def loss(X,Y):
    """损失函数"""
    with tf.name_scope("cost") as scope:
        Hypothesis = inference(X)  
        return tf.reduce_mean(( (Y * tf.log(Hypothesis)) + ((1 - Y) * tf.log(1.0 - Hypothesis)) ) * -1) 
    
        #为啥自己实现了一个交叉熵损失函数？ http://geek.csdn.net/news/detail/126833
        #return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Hypothesis, labels=Y)) 

def train(total_loss):
    """依据 计算的总损失 训练或调整 模型参数"""
    with tf.name_scope("train") as scope:
        learning_rate = 0.05
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

def evaluate(sess,X,Y):
    predicted = tf.cast(inference(X)>0.5,tf.float32)
    print( sess.run( tf.reduce_mean( tf.cast( tf.equal(predicted,Y),tf.float32)) ) )



In [27]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs() 
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator() #??
    threads = tf.train.start_queue_runners(sess=sess, coord=coord) #?
    
    training_steps = 100000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 1000 == 0: 
            print("step:%s, loss:%s " %(step, sess.run([total_loss]))) 
            

    evaluate(sess, X, Y)
    print("step:%s, W1:%s, b1:%s,  W2:%s, b2:%s, loss:%s " %(step,sess.run(Theta1),sess.run(Bias1),sess.run(Theta2),sess.run(Bias2), sess.run([total_loss])))
    print("predicted:%s"% sess.run(inference(X)))
    coord.request_stop()
    coord.join(threads)
    
    #~ tensorboard --logdir="./tmp/graph/xor_0"
    writer = tf.summary.FileWriter('./tmp/graph/xor',sess.graph)
    writer.close()

AttributeError: module 'tensorflow.python.ops.nn' has no attribute 'sigmoid_cross_entropy'